<a href="https://colab.research.google.com/github/pallavmarch/Harry-Potter-LLM-Chatbot/blob/main/HarryPotter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.28.1 trl==0.4.7 pypdf langchain
!pip install --upgrade trl langchain langchain-community gradio -qq

import os
import torch
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, TrainingArguments, pipeline
from trl import SFTTrainer
from datasets import Dataset
from peft import LoraConfig
import locale
import gradio as gr


locale.getpreferredencoding = lambda: "UTF-8"


from google.colab import userdata
sec_key = userdata.get("HF_tkn")
os.environ["HF_tkn"] = sec_key

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.0/110.0 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19

In [ ]:
model_name = "NousResearch/Llama-2-13b-chat-hf"
pdf_path = "/content/J. K. Rowling - Harry Potter 1 - Sorcerer's Stone (4 files merged).pdf"
output_dir = "./results"
num_train_epochs = 3
per_device_train_batch_size = 2
per_device_eval_batch_size = 2  #4 Reduced
gradient_accumulation_steps = 2
learning_rate = 2e-4
max_seq_length = 128
device_map = "auto"

lora_r = 32
lora_alpha = 16
lora_dropout = 0.1

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

loader = PyPDFLoader(pdf_path)
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1200, chunk_overlap=0)
split_documents = text_splitter.split_documents(documents)
split_documents = split_documents[:int(len(split_documents) * 0.5)]    #Select the first 50% of split_documents

dataset = Dataset.from_dict({"text": [doc.page_content for doc in split_documents]})


compute_dtype = getattr(torch, bnb_4bit_compute_dtype)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False



tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [ ]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)


training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    learning_rate=learning_rate,
    fp16=True,
    logging_dir="./logs",
    logging_steps=25
)


In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '0.13.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:300: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:328: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/1419 [00:00<?, ? examples/s]

100%|██████████| 100/100 [00:00<00:00, 39002.27it/s]
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
25,2.224300
50,2.024100
75,1.909400
100,1.981000
125,1.882000
150,1.876200
175,1.842600
200,1.859400
225,1.875900
250,1.849000


TrainOutput(global_step=1065, training_loss=1.7335796857663723, metrics={'train_runtime': 2883.3225, 'train_samples_per_second': 1.476, 'train_steps_per_second': 0.369, 'total_flos': 4.159598988423168e+16, 'train_loss': 1.7335796857663723, 'epoch': 3.0})

In [ ]:
def generate_text(prompt, model, tokenizer, max_length=250):
    instruction = f"[INST] {prompt} [/INST]"
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=max_length,do_sample=True, temperature=0.7)
    result = pipe(instruction)
    return result[0]['generated_text']

In [ ]:
prompt = "Explain me the significance of the Triwizard Tournament in Harry Potter."
generated_text = generate_text(prompt, model, tokenizer)
print(generated_text)

[INST] Explain me the significance of the Triwizard Tournament in Harry Potter. [/INST]  The Triwizard Tournament is a magical competition that takes place every five years between three schools of magic: Hogwarts School of Witchcraft and Wizardry, Beauxbatons Academy of Magic, and Durmstrang Institute. The tournament is significant in the Harry Potter series for several reasons: 
 
1. Dangerous tasks: The Triwizard Tournament involves three dangerous tasks that the champions must complete: the first task is to retrieve a golden egg from underwater, the second is to navigate a maze, and the third is to catch a golden deer. These tasks are extremely dangerous, and the champions are often injured or killed trying to complete them. 
 
2. Age limit: The tournament is open only to students aged seventeen and over, which means that Harry, who is underage, is not eligible to compete. This leads to a lot of tension and conflict in the series, as Harry is forced to watch his friends compete whi

In [ ]:
prompt = "Which challenges does Harry face during the Triwizard Tournament"
generated_text = generate_text(prompt, model, tokenizer)
print(generated_text)

[INST] Which challenges does Harry face during the Triwizard Tournament [/INST]  During the Triwizard Tournament, Harry faces several challenges, including:

1. The First Task: The first challenge is to retrieve a golden egg from underwater in the Black Lake. Harry must retrieve the egg before the other champions, who are all skilled swimmers, and avoid the merpeople guarding the egg.
2. The Second Task: For the second task, Harry must retrieve a golden wand from the Gringotts Wizarding Bank. The task is complicated by the fact that the wand is guarded by a large three-headed dog named Fluffy.
3. The Third Task: The final task is to retrieve a small, golden cup from under the lake's waterfall. The cup is guarded by a giant serpent named Mermaid.
4. The Dark Forest: Harry also faces challenges in the Dark Forest, where he must fight off dementors and other dangers to reach the forest's heart, where he will find and destroy a mysterious and dangerous object.
5. The Maze: Harry must navig

In [ ]:
prompt = "Is Malfoy an ally of Voldemort?"
generated_text = generate_text(prompt, model, tokenizer)
print(generated_text)

[INST] Is Malfoy an ally of Voldemort? [/INST]  Yes, Draco Malfoy is an ally of Lord Voldemort in the Harry Potter series. Malfoy's family has long been associated with the Dark Arts and the evil wizarding world, and Draco himself becomes increasingly involved with the Dark Lord's followers as the series progresses. 

In "Harry Potter and the Goblet of Fire," Malfoy and his friends, Crabbe and Goyle, bully Harry and his friends, and Malfoy's father, Lucius, is revealed to be a Death Eater, a follower of Voldemort. In "Harry Potter and the Order of the Phoenix," Malfoy becomes a prefect at Hogwarts and continues to bully Harry, but also starts to show signs of fear and uncertainty about Voldemort's return. 

In "Harry Potter and the Half-Blood Prince," Malfoy becomes even more involved with the Dark Lord's followers, and is seen at the Hogwarts School of W


In [ ]:
prompt = "What are horcrux?"
generated_text = generate_text(prompt, model, tokenizer)
print(generated_text)

[INST] What are horcrux? [/INST]  In the Harry Potter series, a horcrux is an object that contains a piece of a wizard's soul. This practice of creating horcruxes was started by the dark wizard Salazar Slytherin, one of the four founders of Hogwarts School of Witchcraft and Wizardry. 

To create a horcrux, a wizard must perform a dark and complex ritual, during which he or she must concentrate all his or her evil thoughts and intentions on the object, and then infuse it with a piece of his or her soul. This process is said to be very difficult and requires a great deal of power and skill. 

Once a horcrux is created, it will continue to exist as long as the piece of soul it contains is intact. The horcrux will also be able to sustain the wizard who created it, even if that wizard is injured or weakened. 

The main reason for creating a horcrux is to achieve immortality, as the piece of soul contained within it will


In [ ]:
prompt = "Give me 5 examples of cool potions and explain what they do?"
generated_text = generate_text(prompt, model, tokenizer)
print(generated_text)

[INST] Give me 5 examples of cool potions and explain what they do? [/INST]  Sure, here are five examples of cool potions in the Harry Potter universe and what they do: 
 
1. Felix Felicis (Liquid Luck) 
 
This shimmering silver potion causes the drinker to have the luck they 
deserve for the next few hours. It can make you unexpectedly run into 
good fortune, help you avoid danger, or give you the right words to say 
at the right moment. Felix Felicis can be used once every few weeks, 
and it's a favorite among the Weasley twins. 
 
2. Wolfbane Potion 
 
This potion is used to calm the symptoms of a werewolf during a full moon. 
It can be used to control the violent urges and pain that come with the 
full moon, allowing the werewolf to think and act normally. The 
Wolfbane Potion was invented by Professor Lupin, who himself is a werewolf. 



And

In [ ]:
demo = gr.Interface(
    fn=generate_text,
    inputs=gr.Textbox(label="✨ Enter Your Prompt Here ✨",placeholder="Ask something about Harry Potter...",lines=3,elem_id="input_box"),
    outputs=gr.Textbox(label="🧙‍♂️ Generated Response 🧙‍♀️",lines=10,elem_id="output_box"),
    title="⚡️ Harry Potter LLM Chat ⚡️",
#    css="""
#        #input_box textarea { background-color: #f0f8ff; border: 2px solid #4b0082; color: #333333;font-size: 16px; padding: 10px;}
#        #output_box textarea { background-color: #f5f5f5; border: 2px solid #4b0082; color: #333333; font-size: 16px; padding: 10px;}
#        .gr-button { background-color: #4b0082; color: #ffffff; border: none;font-size: 16px; padding: 10px 20px; border-radius: 5px;}
#        .gr-button:hover { background-color: #6a0dad;}
#    """)
    css="""
            body {
                background-color: #1e1e1e;
                color: #ffffff;
            }
            #input_box textarea {
                background-color: #000000;
                border: 2px solid #4b0082;
                color: #ffffff;
                font-size: 16px;
                padding: 10px;
            }
            #output_box textarea {
                background-color: #000000;
                border: 2px solid #4b0082;
                color: #ffffff;
                font-size: 16px;
                padding: 10px;
            }
            .gr-button {
                background-color: #4b0082;
                color: #ffffff;
                border: none;
                font-size: 16px;
                padding: 10px 20px;
                border-radius: 5px;
            }
            .gr-button:hover {
                background-color: #6a0dad;
                color: #ffffff;
            }
            .container {
                background-color: #1e1e1e;
            }
        """
    )

demo.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://5ffafb484ceae47eb8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
